In [3]:
import threading
from decimal import *
from time import sleep
from datetime import datetime
from uuid import uuid4, UUID
import json
import sqlite3 

from confluent_kafka import SerializingProducer
from confluent_kafka.schema_registry import SchemaRegistryClient
from confluent_kafka.schema_registry.avro import AvroSerializer
from confluent_kafka.serialization import StringSerializer
import mysql.connector
import pandas as pd


def delivery_report(err, msg):
    """
    Reports the failure or success of a message delivery.

    Args:
        err (KafkaError): The error that occurred on None on success.
        msg (Message): The message that was produced or failed.

    Note:
        In the delivery report callback the Message.key() and Message.value()
        will be the binary format as encoded by any configured Serializers and
        not the same object that was passed to produce().
        If you wish to pass the original object(s) for key and value to delivery
        report callback we recommend a bound callback or lambda where you pass
        the objects along.
    """
    if err is not None:
        print("Delivery failed for record {}: {}".format(msg.key(), err))
        return
        print('Record {} successfully produced to {} [{}] at offset {}'.format(
        msg.key(), msg.topic(), msg.partition(), msg.offset()))


# Define Kafka configuration
kafka_config = {
    'bootstrap.servers': 'your_broker_address:9092',  # Replace with your broker's address
    'sasl.mechanisms': 'PLAIN',                        # SASL mechanism
    'security.protocol': 'SASL_SSL',                   # Security protocol
    'sasl.username': 'your_username',                   # SASL username
    'sasl.password': 'your_password'
}

# Create a Schema Registry client
schema_registry_config = {
    'url': 'https://your_schema_registry_url',  # Replace with your Schema Registry URL
    'basic.auth.user.info': 'your_username:your_password'  # Basic auth credentials
}

# SQL connection
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password="password",
  database="databasename"
)

mycursor = mydb.cursor()

# Fetch the latest Avro schema for the value
subject_name = 'topicname'
schema_str = schema_registry_client.get_latest_version(subject_name).schema.schema_str

# Create Avro Serializer for the value
key_serializer = StringSerializer('utf_8')
avro_serializer = AvroSerializer(schema_registry_client, schema_str)

# Define the SerializingProducer
producer = SerializingProducer({
    'bootstrap.servers': kafka_config['bootstrap.servers'],
    'security.protocol': kafka_config['security.protocol'],
    'sasl.mechanisms': kafka_config['sasl.mechanisms'],
    'sasl.username': kafka_config['sasl.username'],
    'sasl.password': kafka_config['sasl.password'],
    'key.serializer': key_serializer,  # Key will be serialized as a string
    'value.serializer': avro_serializer  # Value will be serialized as Avro
})


    
# Load the last read timestamp from the config file
config_data = {}


try:
    with open('config.json', 'r') as f:
        content = f.read()
        print("File content:", content)  # Print the raw content to debug
        config_data = json.loads(content)
        last_read_timestamp = config_data.get('last_read_timestamp')
except FileNotFoundError:
    print("Error: config.json file not found.")
except json.JSONDecodeError as e:
    print(f"Error decoding JSON: {e}")

# Set a default value for last_read_timestamp
if last_read_timestamp is None:
    last_read_timestamp = '1900-01-01 00:00:00'

query= "SELECT * FROM Productdetails WHERE last_updated > '{}'".format(last_read_timestamp)


# Execute the SQL query
mycursor.execute(query)

rows=mycursor.fetchall()
print(rows)
for row in rows:
        # Get the column names from the cursor description
        columns = [column[0] for column in mycursor.description]
        # Create a dictionary from the row values
        value = dict(zip(columns, row))
        # Convert the datetime to string format if last_updated is a datetime object
        if isinstance(value['last_updated'], datetime):
            value['last_updated'] = value['last_updated'].strftime("%Y-%m-%d %H:%M:%S")
        print(value)
        producer.produce(topic='topicname', key=str(value['product_id']), value=value)
        producer.flush()

# Fetch any remaining rows to consume the result
mycursor.fetchall()


#reassigning last_read_timestamp after ones dataLoad
query = "Select Max(last_updated) from Productdetails"

mycursor.execute(query)

result = mycursor.fetchone()

max_read = result[0]
max_read_str = max_read.strftime('%Y-%m-%d %H:%M:%S')
#print(max_read_str)

config_data['last_read_timestamp'] = max_read_str

with open('config.json', 'w') as file:
    json.dump(config_data, file)


# Close the cursor and database connection
mycursor.close()
mydb.close()

print("Data successfully published to Kafka")



File content: {"last_read_timestamp": "2023-07-10 10:13:15"}
[]
Data successfully published to Kafka
